In [15]:
import glob
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import re
import os
import time
from urllib.parse import urlparse
import random
from multiprocessing.pool import ThreadPool
from IPython.display import clear_output

all_txt_files = glob.glob(os.getcwd()+'\\prod_urls\\*.txt')
driver = None
dumb_urls = ['https://www.thepythoncode.com/','https://www.geeksforgeeks.org/','https://pynative.com/','http://facebook.com/','https://www.amazon.in/','https://www.tutorialspoint.com/','https://docs.python.org/','https://chromedriver.chromium.org/','https://github.com/','https://twitter.com/']

home_dir = os.getcwd()
results_dir = home_dir + '\\results'
images_dir = results_dir + '\\images'

In [16]:
ultimate_dataframes = dict()

In [18]:
prod_url_txt_files = [txt_file for txt_file in glob.glob(home_dir+'\\prod_urls\\*.txt')]

genres_and_links = dict()
for filename in prod_url_txt_files:
    with open(filename,'r') as file:
        relative_filename = filename.split('\\')[-1]
        genres_and_links[relative_filename.replace('.txt','')] = file.readlines()
prod_url_txt_files

['E:\\projects-ds-ws\\webscraping-product-details-flipkart\\prod_urls\\Arts, Language and Linguistic Books.txt',
 'E:\\projects-ds-ws\\webscraping-product-details-flipkart\\prod_urls\\Biographies, Memoirs and General Non-Ficton Books.txt',
 'E:\\projects-ds-ws\\webscraping-product-details-flipkart\\prod_urls\\Fiction Books.txt',
 'E:\\projects-ds-ws\\webscraping-product-details-flipkart\\prod_urls\\Healthy Living and Wellness Books.txt',
 'E:\\projects-ds-ws\\webscraping-product-details-flipkart\\prod_urls\\History and Archaeology Books.txt',
 'E:\\projects-ds-ws\\webscraping-product-details-flipkart\\prod_urls\\Literature Books.txt',
 'E:\\projects-ds-ws\\webscraping-product-details-flipkart\\prod_urls\\Philosophy and Religion Books.txt',
 'E:\\projects-ds-ws\\webscraping-product-details-flipkart\\prod_urls\\Self-Help Books.txt']

In [19]:
for kk,vv in genres_and_links.items():
    print(len(vv),'-',kk)

1000 - Arts, Language and Linguistic Books
1000 - Biographies, Memoirs and General Non-Ficton Books
2000 - Fiction Books
1000 - Healthy Living and Wellness Books
1999 - History and Archaeology Books
2000 - Literature Books
1000 - Philosophy and Religion Books
2000 - Self-Help Books


In [6]:
def replace_multispaces_with_single_space(text):
    text = re.sub('\\s+',' ',text)
    return text

In [7]:
def save_image(main_counter,genre,link,driver,all_image_list):
    image_urls = set()
    image_url = None
        
    i_file_name = results_dir + '\\images\\' + genre + '\\' +str(main_counter)+'.jpeg'
    if i_file_name not in all_image_list:
        driver.get(dumb_urls[random.randint(0,len(dumb_urls)-1)])
        driver.get(link)
        page_source = driver.page_source
        soup_ = BeautifulSoup(page_source,'html.parser')
        image_divs = soup_.find_all('div',{'class':'_1ov7-N'})
        for image_div in image_divs:
            img_elements = image_div.find_all('img')
        for img in img_elements:
            if img.has_attr('src'):
                image_url = img.get('src')
        if image_url is not None:
            image_url = re.sub('\\w+?q=\\d+','',image_url)
            image_url = re.sub('https://rukminim1.flixcart.com/image/\\d+/\\d+/','https://rukminim1.flixcart.com/image/512/512/',image_url)
            url_parse_results = urlparse(image_url)
            if all([url_parse_results.scheme, url_parse_results.netloc, url_parse_results.path]):
                image_urls.add(image_url)
        
    if len(image_urls)==0:
        image_url = 'https://img1a.flixcart.com/www/linchpin/fk-cp-zion/img/flipkart-plus_4ee2f9.png'
        image_urls.add(image_url)
            
    i_names = list()
    for i_url in image_urls:
        if i_file_name not in all_image_list:
            requests.get(dumb_urls[random.randint(0,len(dumb_urls)-1)])
            i_response = requests.get(i_url)
            with open(i_file_name, 'wb') as f:
                f.write(i_response.content)
        i_name = i_file_name.split('\\')[-1]
        i_names.append(i_file_name)
        break;
    return i_names;

In [8]:
for genre, links in genres_and_links.items():
    
    genre_images_dir = results_dir + '\\images\\' + genre
    if not os.path.exists(genre_images_dir):
        os.makedirs(genre_images_dir)
    all_image_list = glob.glob(genre_images_dir+'\\*.jpeg')

    main_counter = 1
    for link in links:
        
        if driver is None:
            #options = webdriver.ChromeOptions()
            #options.add_argument("--start-maximized")
            driver = webdriver.Chrome(executable_path="chromedriver.exe")
            #driver = webdriver.Firefox(executable_path='geckodriver.exe')
        
        #pool = ThreadPool(processes=2)
        #async_image_results = pool.apply_async(save_image, (main_counter,genre,link,driver,all_image_list)
        #async_image_results.get()
        save_image(main_counter,genre,link,driver,all_image_list)
                
        print(main_counter)
        if main_counter%50==0:
            clear_output()
            print(genre)
        main_counter += 1
        
    print('yahooooooo!!! Completed for','\"',genre,'\"')

Self-Help Books
yahooooooo!!! Completed for " Self-Help Books "


In [9]:
if driver is not None:
    driver.quit()

In [10]:
print('Everything completed. Thank god.')

Everything completed. Thank god.
